TODO
- mudar o length de products que está errado - OK
- verificar as colunas de relação, que não podem ter null - ok
- Verificar os duplicados de geolocalização
- Fazer o Dataframe principal com as relações

pip:
- notebook
- pandas

# 0 - Preparando o Ambiente

## 0.1 - Download dos CSVs

Com os comandos:
- **curl**: conseguimos baixar o .zip do Kaggle e salvar na pasta brazilian_ecommerce;
- **rm**: caso exista uma pasta com o nome brazilian_ecommerce, apaga ela para não atrapalhar o projeto.
- **unzip**: extrai o .zip que foi baixado o Kaggle

In [47]:
!curl -L -o brazilian-ecommerce.zip https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce
!rm -rf brazilian_ecommerce
!unzip brazilian-ecommerce.zip -d brazilian_ecommerce
!rm brazilian-ecommerce.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 42.6M  100 42.6M    0     0  14.2M      0  0:00:02  0:00:02 --:--:-- 24.2M
Archive:  brazilian-ecommerce.zip
  inflating: brazilian_ecommerce/olist_customers_dataset.csv  
  inflating: brazilian_ecommerce/olist_geolocation_dataset.csv  
  inflating: brazilian_ecommerce/olist_order_items_dataset.csv  
  inflating: brazilian_ecommerce/olist_order_payments_dataset.csv  
  inflating: brazilian_ecommerce/olist_order_reviews_dataset.csv  
  inflating: brazilian_ecommerce/olist_orders_dataset.csv  
  inflating: brazilian_ecommerce/olist_products_dataset.csv  
  inflating: brazilian_ecommerce/olist_sellers_dataset.csv  
  inflating: brazilian_ecommerce/product_category_name_translation.csv  


## 0.2 - Download das bibliotecas necessárias

Usando pip para a instalação das bibliotecas necessárias no decorrer do projeto

In [ ]:
!pip install pandas

# 1 - Preparação dos Dados

## 1.1 - Importações Python

Importando ibliotecas necessárias para a execução completa do notebook

In [55]:
import os
import pandas as pd

## 1.2 - Preparando os DataFrames com os dados dos CSVs

In [56]:
# Diretório que foi criado na etapa de preparação do ambiente 
folder = "brazilian_ecommerce"

# Lista todos os arquivos CSV na pasta
csv_files = [f for f in os.listdir(folder) if f.endswith('.csv')]
dataframe_names = []

# Carrega os DataFrames dinamicamente
for file in csv_files:
    # Remove extensão .csv
    base_name = file.replace('.csv', '')
    # Remove 'olist_', troca 'dataset' por 'df'
    df_name = base_name.replace('olist_', '').replace('dataset', 'df')
    # Lê o CSV
    complete_path = os.path.join(folder, file)
    dataframe_names.append(df_name)
    # Salva em variáveis globais, reaproveitando o nome do arquivo que foi modificado
    globals()[df_name] = pd.read_csv(complete_path)
    print(f"{df_name} loaded with {globals()[df_name].shape[0]} lines.")

sellers_df loaded with 3095 lines.
product_category_name_translation loaded with 71 lines.
orders_df loaded with 99441 lines.
order_items_df loaded with 112650 lines.
customers_df loaded with 99441 lines.
geolocation_df loaded with 1000163 lines.
order_payments_df loaded with 103886 lines.
order_reviews_df loaded with 99224 lines.
products_df loaded with 32951 lines.


## 1.3 - Analisando Dataframes

In [33]:
for df_name in dataframe_names:
    print(f"DataFrame: {df_name}")
    # display(globals()[df_name].head(3))
    print(globals()[df_name].info())
    # print(globals()[df_name].columns)

    # criando um separador entre os dados visualizados
    print()
    print('-'*49)
    print()

DataFrame: sellers_df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB
None

-------------------------------------------------

DataFrame: product_category_name_translation
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
None

-----------------------

Com o **.info()**, podemos verificar características como:
- quantidade de colunas;
- nome das colunas;
- tipo da coluna;
- também contador de dados não nulos.

Ao analisar o nome das colunas, é possível perceber que no dataframe **products_df** existem duas colunas com o nome errado:
- product_name_lenght
- product_description_lenght

Como o correto é **length**, estarei corrigindo o nome:

In [57]:
products_df.rename(columns={
    'product_name_lenght': 'product_name_length',
    'product_description_lenght': 'product_description_length'
}, inplace=True)

products_df.columns

Index(['product_id', 'product_category_name', 'product_name_length',
       'product_description_length', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')

Corrigindo o nome nas colunas, passamos para o próximo passo: as informações **RangeIndex** e **Non-Null Count** dada pelo **info()**. 
Com o **RangeIndex**, observamos o número de linhas de cada coluna. Se o **Non-Null Count** não é igual, significa que alguns valores são nulos. Por isso, é necessária uma análise para saber o que pode ser nulo e o que não pode.

### 🧼 Análise de Colunas com Possíveis Valores Nulos

#### ✅ Objetivo
Identificar quais colunas de cada DataFrame **podem conter valores nulos**, com base no tipo da coluna e na função dela (chave primária, chave estrangeira ou campo informativo).

#### 📌 Regras Gerais

| Tipo de Chave        | Pode ter `null`? | Deve ser Única? |
|----------------------|------------------|------------------|
| **Chave Primária**   | ❌ Nunca          | ✅ Sim           |
| **Chave Estrangeira**| ✅ Sim (às vezes) | ❌ Não           |

#### 🔍 Análise DataFrame por DataFrame

`sellers_df`

| Coluna                  | Pode ter `null`? | Justificativa                     |
|------------------------|------------------|-----------------------------------|
| `seller_id`            | ❌ Não            | Chave primária                    |
| `seller_zip_code_prefix` | ❌ Não          | Localização obrigatória           |
| `seller_city`          | ❌ Não            | Sempre presente                   |
| `seller_state`         | ❌ Não            | Sempre presente                   |

`product_category_name_translation`

| Coluna                          | Pode ter `null`? | Justificativa                     |
|--------------------------------|------------------|-----------------------------------|
| `product_category_name`        | ❌ Não            | Chave primária                    |
| `product_category_name_english`| ❌ Não            | Tradução correspondente           |

`orders_df`

| Coluna                          | Pode ter `null`? | Justificativa                         |
|---------------------------------|------------------|---------------------------------------|
| `order_id`                      | ❌ Não            | Chave primária                        |
| `customer_id`                   | ❌ Não            | FK obrigatória                        |
| `order_status`                  | ❌ Não            | Status sempre presente                |
| `order_purchase_timestamp`      | ❌ Não            | Sempre presente                       |
| `order_approved_at`             | ✅ Sim            | Pode não ter sido aprovado ainda      |
| `order_delivered_carrier_date`  | ✅ Sim            | Pode não ter sido enviado             |
| `order_delivered_customer_date` | ✅ Sim            | Pode não ter sido entregue            |
| `order_estimated_delivery_date` | ❌ Não            | Sempre gerada                         |

`order_items_df`

| Coluna               | Pode ter `null`? | Justificativa                                    |
|----------------------|------------------|--------------------------------------------------|
| `order_id`           | ❌ Não            | Chave estrangeira                                |
| `order_item_id`      | ❌ Não            | Parte da chave primária composta                 |
| `product_id`         | ✅ Sim (raro)     | Pode haver produto não cadastrado                |
| `seller_id`          | ❌ Não            | Sempre presente                                  |
| `shipping_limit_date`| ❌ Não            | Data definida para envio                         |
| `price`              | ❌ Não            | Sempre presente                                  |
| `freight_value`      | ❌ Não            | Sempre presente                                  |

`customers_df`

| Coluna                     | Pode ter `null`? | Justificativa                  |
|----------------------------|------------------|--------------------------------|
| `customer_id`              | ❌ Não            | Chave primária                |
| `customer_unique_id`       | ❌ Não            | Sempre presente               |
| `customer_zip_code_prefix` | ❌ Não            | Localização                   |
| `customer_city`            | ❌ Não            | Sempre presente               |
| `customer_state`           | ❌ Não            | Sempre presente               |

`geolocation_df`

| Coluna                        | Pode ter `null`? | Justificativa                          |
|-------------------------------|------------------|----------------------------------------|
| `geolocation_zip_code_prefix` | ❌ Não            | Usado como referência de localização   |
| `geolocation_lat`             | ❌ Não            | Latitude                                |
| `geolocation_lng`             | ❌ Não            | Longitude                               |
| `geolocation_city`            | ❌ Não            | Sempre presente                         |
| `geolocation_state`           | ❌ Não            | Sempre presente                         |

> ⚠️ Apesar de não ter chave primária formal, essa tabela não deve conter `nulls`.

`order_payments_df`

| Coluna               | Pode ter `null`? | Justificativa              |
|----------------------|------------------|----------------------------|
| `order_id`           | ❌ Não            | Chave estrangeira         |
| `payment_sequential` | ❌ Não            | Parte da chave composta   |
| `payment_type`       | ❌ Não            | Sempre presente           |
| `payment_installments`| ❌ Não           | Sempre presente           |
| `payment_value`      | ❌ Não            | Sempre presente           |

`order_reviews_df`

| Coluna                   | Pode ter `null`? | Justificativa                               |
|---------------------------|------------------|---------------------------------------------|
| `review_id`              | ❌ Não            | Chave primária                              |
| `order_id`               | ❌ Não            | FK obrigatória                              |
| `review_score`           | ❌ Não            | Sempre presente                             |
| `review_comment_title`   | ✅ Sim            | Comentário opcional                         |
| `review_comment_message` | ✅ Sim            | Comentário opcional                         |
| `review_creation_date`   | ❌ Não            | Sempre presente                             |
| `review_answer_timestamp`| ✅ Sim            | Pode não ter sido respondido ainda          |

`products_df`

| Coluna                         | Pode ter `null`? | Justificativa                                  |
|--------------------------------|------------------|------------------------------------------------|
| `product_id`                   | ❌ Não            | Chave primária                                |
| `product_category_name`        | ✅ Sim            | Pode ser desconhecida                          |
| `product_name_length`          | ✅ Sim            | Dados ausentes para alguns produtos            |
| `product_description_length`   | ✅ Sim            | Idem acima                                     |
| `product_photos_qty`           | ✅ Sim            | Idem                                           |
| `product_weight_g`             | ✅ Sim            | Pode estar ausente                             |
| `product_length_cm`            | ✅ Sim            | Idem                                           |
| `product_height_cm`            | ✅ Sim            | Idem                                           |
| `product_width_cm`             | ✅ Sim            | Idem                                           |

Ao verificar cada coluna utilizando **info()**, confirmamos que:

`sellers_df`, `product_category_name_translation`, `order_items_df`, `customers_df`, `geolocation_df`, `order_payments_df` não possuem valores nulos.

`orders_df` possui colunas com valores nulos:

> `order_approved_at`, `order_delivered_carrier_date`, `order_delivered_customer_date`, mas essas colunas podem conter valores nulos

`orders_reviews_df` possui colunas com valores nulos:

> `review_comment_title`, `review_comment_message`, mas essas colunas podem conter valores nulos

`products_df` possui colunas com valores nulos:

> `product_category_name`, `product_name_length`, `product_description_length`, `product_photos_qty`, `product_weight_g`, `product_length_cm`, `product_height_cm`, `product_width_cm`, mas essas colunas podem conter valores nulos

Para confirmar os dados acima, o código abaixo vai remover todos os valores nulos para cada coluna (em cada DataFrame) que não pode ter valores nulos.

In [58]:
# Dicionário com colunas que não podem ter null para cada df
nao_pode_ter_null = {
    'orders_df': ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_estimated_delivery_date'],
    'order_items_df': ['order_id', 'order_item_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value'],
    'customers_df': ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state'],
    'geolocation_df': ['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state'],
    'order_payments_df': ['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value'],
    'order_reviews_df': ['review_id', 'order_id', 'review_score', 'review_creation_date'],
    'products_df': ['product_id'],
    'sellers_df': ['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'],
    'product_category_name_translation': ['product_category_name', 'product_category_name_english']
}

# Limpeza automática
for df_name, colunas in nao_pode_ter_null.items():
    df = globals()[df_name]
    antes = df.shape[0]
    df.dropna(subset=colunas, inplace=True)
    depois = df.shape[0]
    print(f"{df_name}: {antes - depois} linhas removidas devido a valores nulos nas colunas obrigatórias.")


orders_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
order_items_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
customers_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
geolocation_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
order_payments_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
order_reviews_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
products_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
sellers_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
product_category_name_translation: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.


Pela saída percebemos que nenhuma linha foi removida. Mostrando que a verificação feita em utililizando **info()** faz sentido.

## 1.4 - Modificando Tipos

Agora, precisamos resolver o problema de tipagem. Ao observar os dados utilizando **info()**, percebemos que diversas colunas estão com tipo object, principalmente datas.
Vamos verificar novamente os tipos das colunas de cada DataFrame:

In [59]:
# verificando tipos
for df_name in dataframe_names:
    print(f"Dataframe: {df_name}")
    df = globals()[df_name]
    print(df.dtypes)
    print()

Dataframe: sellers_df
seller_id                 object
seller_zip_code_prefix     int64
seller_city               object
seller_state              object
dtype: object

Dataframe: product_category_name_translation
product_category_name            object
product_category_name_english    object
dtype: object

Dataframe: orders_df
order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

Dataframe: order_items_df
order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

Dataframe: customers_df
customer_id                 object
customer_unique_id

Confirmando a diversidade de tipos, o código abaixo deixará os tipos padronizados.

Mas como não queremos perder os NaNs, a melhor abordagem é usar os tipos inteiros nulos (Int64), oferecidos pelo Pandas. Eles preservam os NaNs mesmo em colunas numéricas inteiras.

In [60]:
customers_df = customers_df.astype({
    'customer_id': 'string',
    'customer_unique_id': 'string',
    'customer_zip_code_prefix': 'string',
    'customer_city': 'string',
    'customer_state': 'string'
})

geolocation_df = geolocation_df.astype({
    'geolocation_zip_code_prefix': 'string',
    'geolocation_lat': 'float64',
    'geolocation_lng': 'float64',
    'geolocation_city': 'string',
    'geolocation_state': 'string'
})

order_items_df = order_items_df.astype({
    'order_id': 'string',
    'order_item_id': 'Int64',
    'product_id': 'string',
    'seller_id': 'string',
    'price': 'float64',
    'freight_value': 'float64'
})
order_items_df['shipping_limit_date'] = pd.to_datetime(order_items_df['shipping_limit_date'])

order_payments_df = order_payments_df.astype({
    'order_id': 'string',
    'payment_sequential': 'Int64',
    'payment_type': 'string',
    'payment_installments': 'Int64',
    'payment_value': 'float64'
})

order_reviews_df = order_reviews_df.astype({
    'review_id': 'string',
    'order_id': 'string',
    'review_score': 'Int64',
    'review_comment_title': 'string',
    'review_comment_message': 'string'
})
order_reviews_df['review_creation_date'] = pd.to_datetime(order_reviews_df['review_creation_date'])
order_reviews_df['review_answer_timestamp'] = pd.to_datetime(order_reviews_df['review_answer_timestamp'])

orders_df = orders_df.astype({
    'order_id': 'string',
    'customer_id': 'string',
    'order_status': 'string'
})
orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])
orders_df['order_approved_at'] = pd.to_datetime(orders_df['order_approved_at'])
orders_df['order_delivered_carrier_date'] = pd.to_datetime(orders_df['order_delivered_carrier_date'])
orders_df['order_delivered_customer_date'] = pd.to_datetime(orders_df['order_delivered_customer_date'])
orders_df['order_estimated_delivery_date'] = pd.to_datetime(orders_df['order_estimated_delivery_date'])

products_df = products_df.astype({
    'product_id': 'string',
    'product_category_name': 'string',
    'product_name_length': 'Int64',
    'product_description_length': 'Int64',
    'product_photos_qty': 'Int64',
    'product_weight_g': 'float64',
    'product_length_cm': 'float64',
    'product_height_cm': 'float64',
    'product_width_cm': 'float64'
})

sellers_df = sellers_df.astype({
    'seller_id': 'string',
    'seller_zip_code_prefix': 'string',
    'seller_city': 'string',
    'seller_state': 'string'
})

product_category_name_translation = product_category_name_translation.astype({
    'product_category_name': 'string',
    'product_category_name_english': 'string'
})


Com o código acima executado, vamos verificar se a mudança de tipos foi realizada:

In [61]:
# verificando tipos
for df_name in dataframe_names:
    print(f"Dataframe: {df_name}")
    df = globals()[df_name]
    print(df.dtypes)
    print()

Dataframe: sellers_df
seller_id                 string[python]
seller_zip_code_prefix    string[python]
seller_city               string[python]
seller_state              string[python]
dtype: object

Dataframe: product_category_name_translation
product_category_name            string[python]
product_category_name_english    string[python]
dtype: object

Dataframe: orders_df
order_id                         string[python]
customer_id                      string[python]
order_status                     string[python]
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

Dataframe: order_items_df
order_id               string[python]
order_item_id                   Int64
product_id             string[python]
seller_id              string[python]
shipping_limit_date    datetime64[ns]
price  

Confirmamos que os tipos `object` foram removidos, deixando os tipos mais fáceis de entender e trabalhar.

## 1.5 - Resolvendo Duplicatas

In [64]:
# Verificando duplicatas

for df_name in dataframe_names:
    print(df_name)
    df = globals()[df_name]
    print("Pedidos duplicados:", df.duplicated().sum())

sellers_df
Pedidos duplicados: 0
product_category_name_translation
Pedidos duplicados: 0
orders_df
Pedidos duplicados: 0
order_items_df
Pedidos duplicados: 0
customers_df
Pedidos duplicados: 0
geolocation_df
Pedidos duplicados: 261831
order_payments_df
Pedidos duplicados: 0
order_reviews_df
Pedidos duplicados: 0
products_df
Pedidos duplicados: 0


In [66]:
# verificando quantidade de linhas
geolocation_df.shape[0]

1000163

In [70]:
# quantidade de itens duplicados
geolocation_df[geolocation_df.duplicated()]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
15,1046,-23.546081,-46.644820,sao paulo,SP
44,1046,-23.546081,-46.644820,sao paulo,SP
65,1046,-23.546081,-46.644820,sao paulo,SP
66,1009,-23.546935,-46.636588,sao paulo,SP
67,1046,-23.546081,-46.644820,sao paulo,SP
...,...,...,...,...,...
1000153,99970,-28.343273,-51.873734,ciriaco,RS
1000154,99950,-28.070493,-52.011342,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS


In [5]:
# geolocation é o único com duplicatas
geolocation_df = geolocation_df.drop_duplicates()
geolocation_df.duplicated().sum()

np.int64(0)

In [6]:
# Verificando duplicatas novamente

for df_name in dataframe_names:
    print(df_name)
    df = globals()[df_name]
    print("Pedidos duplicados:", df.duplicated().sum())

sellers_df
Pedidos duplicados: 0
product_category_name_translation
Pedidos duplicados: 0
orders_df
Pedidos duplicados: 0
order_items_df
Pedidos duplicados: 0
customers_df
Pedidos duplicados: 0
geolocation_df
Pedidos duplicados: 0
order_payments_df
Pedidos duplicados: 0
order_reviews_df
Pedidos duplicados: 0
products_df
Pedidos duplicados: 0


In [ ]:
nulls_in_product_name_length = products_df['product_name_length'].isnull()

products_df[nulls_in_product_name_length]

In [ ]:
nulos = products_df['product_name_length'].isnull()

products_df[nulls_in_product_name_length]

In [19]:
for df_name in dataframe_names:
    df = globals()[df_name]
    null_counts = df.isnull().sum()
    null_cols = null_counts[null_counts > 0]
    if not null_cols.empty:
        print(f"\nDataFrame: {df_name}")
        for col, count in null_cols.items():
            print(f"  Coluna '{col}' tem {count} valores nulos.")
    else:
        print(f"\nDataFrame: {df_name} - Nenhum valor nulo encontrado.")



DataFrame: sellers_df - Nenhum valor nulo encontrado.

DataFrame: product_category_name_translation - Nenhum valor nulo encontrado.

DataFrame: orders_df
  Coluna 'order_approved_at' tem 160 valores nulos.
  Coluna 'order_delivered_carrier_date' tem 1783 valores nulos.
  Coluna 'order_delivered_customer_date' tem 2965 valores nulos.

DataFrame: order_items_df - Nenhum valor nulo encontrado.

DataFrame: customers_df - Nenhum valor nulo encontrado.

DataFrame: geolocation_df - Nenhum valor nulo encontrado.

DataFrame: order_payments_df - Nenhum valor nulo encontrado.

DataFrame: order_reviews_df
  Coluna 'review_comment_title' tem 87656 valores nulos.
  Coluna 'review_comment_message' tem 58247 valores nulos.

DataFrame: products_df
  Coluna 'product_category_name' tem 610 valores nulos.
  Coluna 'product_name_lenght' tem 610 valores nulos.
  Coluna 'product_description_lenght' tem 610 valores nulos.
  Coluna 'product_photos_qty' tem 610 valores nulos.
  Coluna 'product_weight_g' tem 2 

In [ ]:
for df_name in dataframe_names:
    df = globals()[df_name]

    if df.duplicated().any():
        print(f"DataFrame '{df_name}' tem duplicatas. Removendo...")
        df = df.drop_duplicates()
        globals()[df_name] = df

In [ ]:
geolocation_df#.head(3)
# geolocation_df.isnull().sum()

In [ ]:
order_items_df.head(3)

In [ ]:
order_payments_df.head(3)


In [ ]:
order_reviews_df.head(3)

In [ ]:
orders_df.head(3)

In [ ]:
products_df.head(3)

In [ ]:
sellers_df.head(3)

In [ ]:
product_category_name_translation.head(5)